In [19]:
import re
import pandas as pd
from nltk.corpus import stopwords

import numpy as np

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE 
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [20]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\Negative_News.xlsx")
news.head()

,Title,Body,Category,Old MegaCategory,MegaCategory,important
0,Massive fire breaks out at Asian Paints manufa...,Click here to know more.,Black Swan Events - Alert me when a company fa...,Negative News,Negative News,Major Loss for the Company
1,BPCL refinery fire doused but tension continue...,BPCL is in the process of setting up a committ...,Black Swan Events - Alert me when a company fa...,Negative News,Negative News,Major Loss for the Company
2,BPCL blaze extinguished; sleepless night for l...,Mumbai: The fire at the Mahul refinery of the ...,Black Swan Events - Alert me when a company fa...,Negative News,Negative News,Major Loss for the Company
3,Explosion in BPCL plant in Chembur; 43 injured,There was a major explosion on Wednesday in th...,Black Swan Events - Alert me when a company fa...,Negative News,Negative News,Major Loss for the Company
4,45 injured as fire breaks out at BPCL refinery...,The fire was brought under control after three...,Black Swan Events - Alert me when a company fa...,Negative News,Negative News,Major Loss for the Company


In [21]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [22]:
news['text'] = news.Title + " " + news.Body

In [23]:
# import these modules 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("shareholders :", lemmatizer.lemmatize("shareholder")) 
print("stocks :", lemmatizer.lemmatize("stock")) 
print("companys :", lemmatizer.lemmatize("company")) 
print("trading :", lemmatizer.lemmatize("trade")) 
print("weekly :", lemmatizer.lemmatize("week")) 
print("billion:", lemmatizer.lemmatize("billionaire")) 
print("high :", lemmatizer.lemmatize("higher")) 
print("close :", lemmatizer.lemmatize("closeed")) 
print("finance :", lemmatizer.lemmatize("financial")) 
print("biggest:", lemmatizer.lemmatize("high")) 
print("unit:", lemmatizer.lemmatize("units")) 
print("protect:", lemmatizer.lemmatize("protection")) 
print("organisation:", lemmatizer.lemmatize("organised")) 
print("'company:", lemmatizer.lemmatize("'companies")) 
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a"))

shareholders : shareholder
stocks : stock
companys : company
trading : trade
weekly : week
billion: billionaire
high : higher
close : closeed
finance : financial
biggest: high
unit: unit
protect: protection
organisation: organised
'company: 'companies
better : good


In [24]:
# import these modules 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("shareholders :", lemmatizer.lemmatize("shareholder")) 
print("stocks :", lemmatizer.lemmatize("stock")) 
print("companys :", lemmatizer.lemmatize("company")) 
print("trading :", lemmatizer.lemmatize("trade")) 
print("weekly :", lemmatizer.lemmatize("week")) 
print("billion:", lemmatizer.lemmatize("billionaire")) 
print("high :", lemmatizer.lemmatize("higher")) 
print("close :", lemmatizer.lemmatize("closeed")) 
print("finance :", lemmatizer.lemmatize("financial")) 
print("biggest:", lemmatizer.lemmatize("high")) 
print("unit:", lemmatizer.lemmatize("units")) 
print("protect:", lemmatizer.lemmatize("protection")) 
print("organisation:", lemmatizer.lemmatize("organised")) 
print("'company:", lemmatizer.lemmatize("'companies")) 
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a"))

shareholders : shareholder
stocks : stock
companys : company
trading : trade
weekly : week
billion: billionaire
high : higher
close : closeed
finance : financial
biggest: high
unit: unit
protect: protection
organisation: organised
'company: 'companies
better : good


In [25]:
 news['text']=news['text'].apply(lemmatizer.lemmatize)

In [26]:

news['clean_title'] = news['Title'].apply(remove_stop_words)
news['clean_body'] = news['Body'].apply(remove_stop_words)

news['clean_text'] = news['text'].apply(remove_stop_words) 

#remove_stop_words_Body


In [27]:
# ## Split data
print("\nSplitting data")

title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_text'], news.important,test_size =.1)
title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.1)


print("Training: ",len(title_tr))

print("Testing: ",len(title_te))


Splitting data
Training:  175
Testing:  22


In [28]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [29]:
print(Ytr)

[1 4 2 4 2 3 2 4 2 2 0 3 3 4 1 0 2 0 2 2 2 2 4 3 2 3 2 1 2 2 3 1 1 3 2 4 4
 4 0 0 4 2 3 0 4 4 4 4 4 0 1 3 3 3 4 4 1 2 2 3 2 3 3 4 4 4 2 1 2 4 0 3 0 0
 2 4 1 0 4 1 1 2 4 3 3 4 2 3 2 3 0 4 3 2 0 1 3 2 3 1 1 4 4 1 4 3 4 3 0 3 4
 4 3 1 4 2 0 2 4 0 1 4 3 2 1 4 3 0 4 3 4 4 1 2 3 1 0 0 2 2 3 2 0 3 2 2 2 1
 2 2 2 2 2 3 0 3 0 2 2 4 4 3 1 3 4 0 0 3 4 2 2 3 1 0 4]


In [30]:
Xtr

<175x3558 sparse matrix of type '<class 'numpy.int64'>'
	with 9620 stored elements in Compressed Sparse Row format>

In [31]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)

dc_pred = dc.predict(Xde)



Training baseline classifier


In [32]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                            precision    recall  f1-score   support

       Financial Challenge       0.00      0.00      0.00         4
                   Layoffs       0.00      0.00      0.00         6
Major Loss for the Company       0.33      0.50      0.40         2
 Regulatory or Legal Issue       0.00      0.00      0.00         6
                    others       0.33      0.50      0.40         2

                  accuracy                           0.16       707
                 macro avg       0.15      0.22      0.18       707
              weighted avg       0.11      0.15      0.12       707

Accuracy achieved is 0.15


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))


In [33]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                            precision    recall  f1-score   support

       Financial Challenge       0.67      1.00      0.80         4
                   Layoffs       0.40      0.33      0.36         6
Major Loss for the Company       0.67      1.00      0.80         2
 Regulatory or Legal Issue       0.40      0.33      0.36         6
                    others       0.67      1.00      0.80         2

                  accuracy                           0.49       707
                 macro avg       0.47      0.56      0.50       707
              weighted avg       0.45      0.49      0.46       707

Accuracy achieved is 0.5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))


In [34]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                            precision    recall  f1-score   support

       Financial Challenge       1.00      0.50      0.67         4
                   Layoffs       0.46      1.00      0.63         6
Major Loss for the Company       1.00      1.00      1.00         2
 Regulatory or Legal Issue       0.46      1.00      0.63         6
                    others       1.00      1.00      1.00         2

                  accuracy                           0.62       707
                 macro avg       0.65      0.66      0.61       707
              weighted avg       0.61      0.67      0.58       707

Accuracy achieved is 0.6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                            precision    recall  f1-score   support

       Financial Challenge       0.57      1.00      0.73         4
                   Layoffs       1.00      0.67      0.80         6
Major Loss for the Company       1.00      1.00      1.00         2
 Regulatory or Legal Issue       1.00      0.67      0.80         6
                    others       1.00      1.00      1.00         2

                  accuracy                           0.73       707
                 macro avg       0.79      0.74      0.75       707
              weighted avg       0.81      0.70      0.73       707

Accuracy achieved is 0.7


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))


In [36]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

Training Support Vector Classification
                            precision    recall  f1-score   support

       Financial Challenge       0.00      0.00      0.00         4
                   Layoffs       0.35      1.00      0.52         6
Major Loss for the Company       0.67      1.00      0.80         2
 Regulatory or Legal Issue       0.35      1.00      0.52         6
                    others       0.67      1.00      0.80         2

                  accuracy                           0.44       707
                 macro avg       0.26      0.51      0.34       707
              weighted avg       0.22      0.50      0.29       707

Accuracy achieved is 0.4


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [37]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                            precision    recall  f1-score   support

       Financial Challenge       1.00      1.00      1.00         4
                   Layoffs       0.71      0.83      0.77         6
Major Loss for the Company       1.00      1.00      1.00         2
 Regulatory or Legal Issue       0.71      0.83      0.77         6
                    others       1.00      1.00      1.00         2

                  accuracy                           0.76       707
                 macro avg       0.78      0.78      0.78       707
              weighted avg       0.76      0.76      0.76       707

Accuracy achieved is 0.75


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 175, does not match size of target_names, 5
  .format(len(labels), len(target_names))


In [38]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.15
Accuracy achieved is 0.5
Accuracy achieved is 0.7
Accuracy achieved is 0.4
Accuracy achieved is 0.75


In [39]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                            precision    recall  f1-score   support

       Financial Challenge       1.00      0.60      0.75         5
                   Layoffs       0.38      0.75      0.50         4
Major Loss for the Company       1.00      0.83      0.91         6
 Regulatory or Legal Issue       0.38      0.75      0.50         4
                    others       1.00      0.83      0.91         6

                  accuracy                           0.56       816
                 macro avg       0.54      0.52      0.50       816
              weighted avg       0.63      0.57      0.57       816

Accuracy achieved is 0.5454545454545454


In [40]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [41]:
#df = pd.DataFrame(output, columns=["title","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")

In [42]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"text":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["text","predicted","actual"])


Accuracy achieved is 0.5454545454545454


In [43]:
df = pd.DataFrame(output, columns=["text","predicted","actual"])
df.to_csv("C:\\Users\\Kratika\\Downloads\\old.csv")

In [44]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(news.clean_text)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))


In [46]:
from sklearn.neural_network import MLPClassifier

#mlp =       MLPClassifier(solver='adam',  alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, news.important, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [47]:
predicted = clf_neural.predict(X_test)
print(classification_report(y_test, predicted))
print('Accuracy achieved is ' + str(np.mean(predicted == y_test)))


                            precision    recall  f1-score   support

       Financial Challenge       0.67      0.50      0.57         8
                   Layoffs       0.67      0.89      0.76         9
Major Loss for the Company       0.92      0.92      0.92        13
 Regulatory or Legal Issue       1.00      0.56      0.72        16
                    others       0.33      0.56      0.42         9

                  accuracy                           0.69        55
                 macro avg       0.72      0.69      0.68        55
              weighted avg       0.77      0.69      0.70        55

Accuracy achieved is 0.6909090909090909


In [48]:
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print(classification_report(y_test, mlp_pred))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == y_test)))

                            precision    recall  f1-score   support

       Financial Challenge       0.33      0.12      0.18         8
                   Layoffs       0.89      0.89      0.89         9
Major Loss for the Company       0.87      1.00      0.93        13
 Regulatory or Legal Issue       0.67      0.62      0.65        16
                    others       0.23      0.33      0.27         9

                  accuracy                           0.64        55
                 macro avg       0.60      0.59      0.58        55
              weighted avg       0.63      0.64      0.62        55

Accuracy achieved is 0.6363636363636364


In [49]:
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100,), activation='relu', random_state=1, max_iter=400)

mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)  
print(classification_report(y_test, mlp_pred))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == y_test)))

                            precision    recall  f1-score   support

       Financial Challenge       0.50      0.38      0.43         8
                   Layoffs       0.89      0.89      0.89         9
Major Loss for the Company       0.76      1.00      0.87        13
 Regulatory or Legal Issue       0.71      0.62      0.67        16
                    others       0.11      0.11      0.11         9

                  accuracy                           0.64        55
                 macro avg       0.60      0.60      0.59        55
              weighted avg       0.62      0.64      0.62        55

Accuracy achieved is 0.6363636363636364
